In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import FL_utils.FL_interpretable_functions as interpret
import FL_utils.FL_RiskSLIM as slim
import FL_utils.FL_stumps as stumps
import FL_utils.FL_Corel as Corel

# restore saved variables
%store -r summary_property2_fl_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\FL Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_two_year'].values

In [3]:
#### CART
depth = [1,2,3]
impurity = [0.001, 0.003]
split = [2,3,4]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, split, impurity, seed=816)

### EBM
estimators = [60,80,100]
depth = [1,2]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, holdout_split, seed=816)

In [7]:
cart_summary['KY_score'], ebm_summary['KY_score']

(0.5218770496363779, 0.6606040351991844)

### Lasso Stumps

In [9]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [21]:
c_grid = {'C':[0.01, 0.02, 0.03, 0.04, 0.05]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [22]:
stumps_summary

{'best_auc': 0.7295905771739511,
 'best_params': {'C': 0.05},
 'auc_diffs': 0.0403763634396298,
 'KY_score': 0.5448911018500571}

### RiskSLIM

In [34]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [35]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.03, random_state=816).fit(FL_X, FL_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

14

In [36]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_FL_X.insert(0, '(Intercept)', 1)

In [37]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'property_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=1000, 
                                max_offset = 100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/20/19 @ 11:29 AM | 193 rows in lookup table
10/20/19 @ 11:29 AM | ------------------------------------------------------------
10/20/19 @ 11:29 AM | runnning initialization procedure
10/20/19 @ 11:29 AM | ------------------------------------------------------------
10/20/19 @ 11:29 AM | CPA produced 2 cuts
10/20/19 @ 11:29 AM | running naive rounding on 6 solutions
10/20/19 @ 11:29 AM | best objective value: 0.5620
10/20/19 @ 11:29 AM | rounding produced 5 integer solutions
10/20/19 @ 11:29 AM | best objective value is 0.4544
10/20/19 @ 11:29 AM | running sequential rounding on 6 solutions
10/20/19 @ 11:29 AM | best objective value: 0.5620
10/20/19 @ 11:29 AM | sequential rounding produced 1 integer solutions
10/20/19 @ 11:29 AM | best objective value: 0.4544
10/20/19 @ 11:29 AM | polishing 6 solutions
10/20/19 @ 11:29 AM | best objective value: 0.4544
10/20/19 @ 11:29 AM | polishing produced 5 integer solutions
10/20/19 @ 1

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:29 AM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2752.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 144100 10220        cutoff              0.2673        0.2648   758722    0.93%           rho_6 U 144100 137398     36
 147407  8869        cutoff              0.2673        0.2650   770962    0.83%           rho_6 D 147407 147405     23
Elapsed time = 41.95 sec. (41259.18 ticks, tree = 3.27 MB, solutions = 8)
 150724  7370        cutoff              0.2673        0.2654   782550    0.71%           rho_5 U 150724 150721     25
 153957  5579        cutoff              0.2673        0.2657   793787    0.57%           rho_3 U 153957  95796     28
 157176  3495        cutoff              0.2673        0.2662   803666    0.39%           rho_3 N 157176  70160     24
 160374   897        cutoff              0.2673        0.2669   811665    0.12%          rho_10 U 160374 139606     22

User cuts applied:  436

Root node processing (before b&c):
  Real time             =    0.11 sec. (1.45 ticks)
Sequential b&c:
  Real time             =   44.94 sec. (45342.87 ticks)
                          -

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:30 AM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2855.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

10/20/19 @ 11:30 AM | ------------------------------------------------------------
10/20/19 @ 11:30 AM | runnning initialization procedure
10/20/19 @ 11:30 AM | ------------------------------------------------------------
10/20/19 @ 11:30 AM | CPA produced 2 cuts
10/20/19 @ 11:30 AM | running naive rounding on 5 solutions
10/20/19 @ 11:30 AM | best objective value: 0.6162
10/20/19 @ 11:30 AM | rounding produced 5 integer solutions
10/20/19 @ 11:30 AM | best objective value is 0.6357
10/20/19 @ 11:30 AM | running sequential rounding on 5 solutions
10/20/19 @ 11:30 AM | best objective value: 0.6162
10/20/19 @ 11:30 AM | sequential rounding produced 1 integer solutions
10/20/19 @ 11:30 AM | best objective value: 0.4630
10/20/19 @ 11:30 AM | polishing 6 solutions
10/20/19 @ 11:30 AM | best objective value: 0.4630
10/20/19 @ 11:30 AM | polishing produced 4 integer solutions
10/20/19 @ 11:30 AM | initialization produced 9 feasible solutions
10/20/19 @ 11:30 AM | best objective value: 0.2922


...

 125399 10465        cutoff              0.2846        0.2814   712286    1.12%           rho_2 D 125399  51893     34
 128348  9533        cutoff              0.2846        0.2817   724065    1.02%           rho_6 U 128348 128346     28
Elapsed time = 41.92 sec. (41261.75 ticks, tree = 2.83 MB, solutions = 9)
 131258  8501        0.2830     9        0.2846        0.2820   735625    0.91%           rho_4 N 131258  32623     31
 134095  7234        cutoff              0.2846        0.2824   746647    0.79%          rho_11 U 134095   1732     19
 136922  5851        0.2827    14        0.2846        0.2827   756634    0.66%           rho_0 D 136922 129421     42
 139758  4264        cutoff              0.2846        0.2832   765974    0.51%          rho_11 D 139758  65087     23
 142570  2388        cutoff              0.2846        0.2837   774297    0.32%           rho_9 U 142570  13380     32

Gomory fractional cuts applied:  1
User cuts applied:  534

Root node processing (before b&c

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:31 AM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2713.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:32 AM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2907.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

 127511 31288        0.2734     7        0.2767        0.2687   707857    2.90%           rho_0 U 127511 127510     32
 130588 31248        cutoff              0.2767        0.2689   722316    2.84%           rho_2 D 130588  70777     32
 133669 31159        cutoff              0.2767        0.2690   736709    2.78%           rho_7 N 133669 133667     22
 136754 31063        0.2703    15        0.2767        0.2692   751144    2.72%           rho_0 D 136754  76529     19
 139805 30811        cutoff              0.2767        0.2693   765389    2.67%           rho_1 N 139805  76366     20
 142880 30669        0.2716    12        0.2767        0.2695   779382    2.61%           rho_2 N 142880  34119     19
 145907 30421        cutoff              0.2767        0.2697   793581    2.55%           rho_0 N 145907  41029     24
Elapsed time = 42.92 sec. (41258.56 ticks, tree = 9.42 MB, solutions = 16)
 148930 30184        0.2730     9        0.2767        0.2698   807634    2.50%           rh

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/20/19 @ 11:33 AM | adding 254 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2844.
Tried aggregator 1 time.
Reduced MIP has 30 rows, 32 columns, and 87 nonzeros.
Reduced MIP has 14 binaries, 16 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.04 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [38]:
riskslim_summary['KY_score'], np.mean(riskslim_summary['FL_train_score'])

(0.539310419403703, 0.7340884057271854)

In [31]:
#### save results
summary_property2_fl_inter_model = {"CART": cart_summary,
                                    "EBM": ebm_summary,
                                    "Stumps": stumps_summary,
                                    "RiskSLIM": riskslim_summary}
%store summary_property2_fl_inter_model

Stored 'summary_property2_fl_inter_model' (dict)


In [32]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\FL Model\\interpretable\\Two Year\\"
results = [["Property",             
            round(cart_summary['KY_score'],3), 
            round(ebm_summary['KY_score'], 3), 
            round(stumps_summary['KY_score'],3), 
            round(riskslim_summary['KY_score'], 3)]]
with open(path + 'Two Year FL_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)